In [1]:
from transformers import GPT2Tokenizer
import os.path

In [ ]:
import sys
sys.path.insert(0, '/u/scr/katezhou/benchmarking/')
from utils import *

In [15]:
%reset
%run /u/scr/katezhou/benchmarking/utils.ipynb

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
{'gpt3': {'daily': Usage(period='2023-8-23', used=1000362, quota=None), 'monthly': Usage(period='2023-8', used=1000362, quota=None), 'total': Usage(period='all', used=6000057, quota=6000000)}, 'codex': {'daily': Usage(period=None, used=0, quota=10000)}, 'jurassic': {'daily': Usage(period=None, used=0, quota=10000)}, 'gooseai': {'daily': Usage(period=None, used=0, quota=10000)}, 'cohere': {'daily': Usage(period=None, used=0, quota=10000)}, 'text_to_image': {'daily': Usage(period=None, used=0, quota=10)}, 'dall_e': {'daily': Usage(period=None, used=0, quota=5)}, 'together_vision': {'daily': Usage(period=None, used=0, quota=30)}, 'gpt': {'daily': Usage(period=None, used=0, quota=None), 'monthly': Usage(period=None, used=0, quota=None)}, 'gpt4': {'daily': Usage(period='2023-8-23', used=74900, quota=None), 'monthly': Usage(period='2023-8', used=74900, quota=None), 'total': Usage(period='all', used=415714, quota=1000000)}}


In [3]:
class dataset():
    
    def __init__(self, name, sample_questions_files):
        self.name = name
        self.sample_questions = pd.DataFrame()

        for sample_questions_file in sample_questions_files: 
            self.sample_questions = pd.concat([self.sample_questions, pd.read_csv(sample_questions_file)])
         
        if name == 'triviaqa':
            self.sample_questions = self.sample_questions[['Question_example', 'Answer.MatchedWikiEntityName_example', 'Question', 'Answer.Aliases']]
        elif name == 'countryqa':
            self.sample_questions = self.sample_questions[['name_example', 'capital_example', 'name', 'capital']]
        else:
            self.sample_questions = self.sample_questions[['question_example', 'answer_example', 'question', 'answer']]
            
        self.sample_questions.columns = ['question_example', 'answer_example', 'question', 'answer']
        
        if name == 'boolqa':
            self.sample_questions['answer'] =  self.sample_questions['answer'].astype(str)
            self.sample_questions['answer_example'] =  self.sample_questions['answer_example'].astype(str)


In [4]:
def filter_one_token_responses(df, answer_column_name):
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    #tokenize all words using GPT2's tokenizer
    df['tokens'] = df[answer_column_name].apply(lambda x: len(tokenizer(" " + str(x))['input_ids']))
    df = df[df['tokens']==1]
    #length of single token answer
    print("Total number of one token answer examples:", len(df))
    return df

# Querying and checking answers

In [5]:
def token_processing(token):
    token_prob_mapping = token.top_logprobs
    token_prob_mapping = {k: v for k, v in sorted(token_prob_mapping.items(), key=lambda item: item[1], reverse=True)}

    topk_predictions = list(token_prob_mapping.keys())
    topk_predictions_log_prob = list(token_prob_mapping.values())

    return topk_predictions, topk_predictions_log_prob

In [6]:
def check_gold_answer(topk_predictions, topk_predictions_log_prob, answer, name):
    #   if the actual answer is in the top 5 predictions
    topk_predictions = [x.strip(" ").lower() for x in topk_predictions]
    gold_answer_logprob = 0

    for index, prediction in enumerate(topk_predictions):
        if (name == 'triviaqa'):
            if (prediction in answer):
                #Sum together all possible correct answers
                gold_answer_logprob += np.e**(topk_predictions_log_prob[index])
                print("one answer", prediction, gold_answer_logprob)
        else:
            if (prediction == answer[0]):
                #Sum together all possible correct answers
                gold_answer_logprob += np.e**(topk_predictions_log_prob[index])
                print("one answer", prediction, gold_answer_logprob)

    print("final prob", gold_answer_logprob)
    return gold_answer_logprob


In [7]:
def building_prompts(current_df, name, TOTAL_QUESTIONS_TO_QUERY):
    prompts_to_use = pd.DataFrame()
    #go through all the example prompts
    for n, row in current_df.sample_questions[:TOTAL_QUESTIONS_TO_QUERY].iterrows():
        
        question = row['question']
            
        answer = [x.strip("\'") for x in row['answer'].lower().strip('][').split(', ')]
        
        if current_df.name == 'countryqa' or current_df.name == 'triviaqa':
            prompt = "Q: " + question + "\nA:" + name           
        else:
            prompt = "Q: " + question + " A:" + name

        prompts_to_use = pd.concat([prompts_to_use, pd.DataFrame([[prompt, 
                                                                   question, 
                                                                   answer]])])
    prompts_to_use.columns = ['prompt', 
                           'question', 
                           'answer']
    return prompts_to_use

In [8]:
def query_api(prompts_to_use, current_df, name, TOTAL_QUESTIONS_TO_QUERY, TEMPERATURE, MODEL_NAME, NUM_COMPLETIONS, STOP_SEQUENCES, ECHO_PROMPT, TOP_K_PER_TOKEN, MAX_TOKENS):
    
    for n, row in prompts_to_use[:TOTAL_QUESTIONS_TO_QUERY].iterrows():
        question = row['question']
        answer = row['answer']
        prompt = row['prompt']
        answer = row['answer']

        print("prompt", prompt)
        print("answer", answer)
        print("----")
        
        request = Request(prompt=prompt, temperature=TEMPERATURE, model=MODEL_NAME, num_completions=NUM_COMPLETIONS, max_tokens=MAX_TOKENS, stop_sequences=STOP_SEQUENCES, echo_prompt=ECHO_PROMPT, top_k_per_token=TOP_K_PER_TOKEN)
        request_result: RequestResult = service.make_request(auth, request)

        print("hello")
        #one completion
        sequence = request_result.completions[0]
        text = [x.text for x in sequence.tokens]
        print(text)

        #make sure at least one token was generated
        if len(sequence.tokens) == 0:
            current_responses = pd.DataFrame([[prompt,
                                               question, 
                                               answer,
                                               -1,
                                               -1,
                                               "",
                                               name] +
                                               [] +
                                               []])
        
        else:
        
            topk_predictions_answer_list = [[]] * MAX_TOKENS
            topk_predictions_log_prob_answer_list = [[]] * MAX_TOKENS

            best_token_index = 0
            best_token_value = -10000
            for n, token in enumerate(sequence.tokens):
                topk_predictions_answer, topk_predictions_log_prob_answer = token_processing(token)
                topk_predictions_answer_list[n] = topk_predictions_answer
                topk_predictions_log_prob_answer_list[n] = topk_predictions_log_prob_answer

                gold_answer_logprob_answer = check_gold_answer(topk_predictions_answer, topk_predictions_log_prob_answer, answer, current_df.name)
                if (gold_answer_logprob_answer > best_token_value):
                    best_token_value = gold_answer_logprob_answer
                    best_token_index = n

            current_responses = pd.DataFrame([[prompt,
                                               question, 
                                               answer,
                                               best_token_value,
                                               best_token_index,
                                               text,
                                               name] +
                                               topk_predictions_answer_list +
                                               topk_predictions_log_prob_answer_list])
        
        current_responses.to_csv(output_file, mode='a', header=False, index=False)

In [9]:
def get_datasets(folder_path, dataset_name):
    
    if dataset_name == 'countryqa':
        df = dataset('countryqa', [folder_path + "countryqa/data/all_questions.csv"])
        df.sample_questions = filter_one_token_responses(df.sample_questions, 'answer')
    else:
        df = dataset(dataset_name, [folder_path + dataset_name + "/data/sample_questions_single_token_answers_50.csv",
                                       folder_path + dataset_name + "/data/sample_questions_single_token_answers_50_2.csv",
                                      folder_path +  dataset_name + "/data/sample_questions_single_token_answers_100.csv"])
    print("number of questions read in", len(df.sample_questions))
    return df


# Read in data

In [10]:
def display_results():
    queried = pd.read_csv(output_file, header=None).drop_duplicates()
    queried.columns = ['prompt', 
                       'question', 
                       'answer',
                       'best_gold_answer_logprob_answer',
                       'best_token_index',
                       'text',
                       'name'] + ['topk_predictions_answer_' + str(x) for x in range(0, 10)] + ['topk_predictions_log_prob_answer_' + str(x) for x in range(0, 10)]

    print("output file", output_file)

    queried['name'] = queried['name'].fillna("unprompted")
    queried = queried[queried['name'] != "It's"]
    # queried = queried.dropna(subset='topk_predictions_answer_0')
    display(queried.groupby("name").mean())
    



In [11]:
def main(dataset_name, model_name):
    folder_path = '/u/scr/katezhou/Uncertainty/interpretability/'

    TOTAL_QUESTIONS_TO_QUERY = 200
    NUM_COMPLETIONS = 1
    MAX_TOKENS = 10
    TEMPERATURE = 1
    TOP_K_PER_TOKEN = 50
    ECHO_PROMPT = False
    STOP_SEQUENCES = []

    prefix_df = pd.read_csv(folder_path + "uncertainty_template_meta.csv")
    PREFIXES = [""] + [" " + x for x in  prefix_df['name'].values]
    PREFIXES.remove(" unprompted")
    PREFIXES.remove( " I don't think it's")
    PREFIXES.remove(" I seriously doubt it's")
    print("number of prefixes", len(PREFIXES))

    current_df = get_datasets(folder_path, dataset_name)
    MODEL_NAME = "openai/" + model_name
    print("dataset", dataset_name, "model name", model_name)

    if (current_df.name == 'jeopardy'):
        name = "gricean_experiment_qa_cleaned"
        current_df.sample_questions['question_example'] = current_df.sample_questions['question_example'].apply(lambda x: x.replace("<br />", " "))
        current_df.sample_questions['question'] = current_df.sample_questions['question'].apply(lambda x: x.replace("<br />", " "))
    else:
        name = "gricean_experiment_qa"

    path = current_df.name + '/results/'
    output_file = folder_path + path + str(MAX_TOKENS) + "_" + str(TEMPERATURE)[-1:] + "_" + MODEL_NAME.replace("/", "_") + "_" + name + '.csv'

    print("output_file", output_file)

    queried = pd.DataFrame()
    queried_prompts = set()
    if os.path.exists(output_file):

        queried = pd.read_csv(output_file, header=None).drop_duplicates()
        queried.columns = ['prompt',
                           'question', 
                           'answer',
                           'best_gold_answer_logprob_answer',
                           'best_token_index',
                           'text',
                           'name'] + ['topk_predictions_answer_' + str(x) for x in range(0, 10)] + ['topk_predictions_log_prob_answer_' + str(x) for x in range(0, 10)]
        queried_prompts = set(queried['prompt'].to_list())

    total_prompts_to_query = 0
    print()
    for name in PREFIXES:
        prompts_to_use = building_prompts(current_df, name, TOTAL_QUESTIONS_TO_QUERY)
        prompts_set = set(prompts_to_use['prompt'].to_list())

        print("template:", name)
        print('len prompts to use', len(prompts_set))
        print('already queried', len(queried_prompts.intersection(prompts_set)))
        prompts_set = prompts_set - queried_prompts
        print('prompts to query', len(prompts_set))
        total_prompts_to_query += len(prompts_set)
    #     query_api(prompts_to_use[prompts_to_use['prompt'].isin(prompts_set)], current_df, name, TOTAL_QUESTIONS_TO_QUERY, TEMPERATURE, MODEL_NAME, NUM_COMPLETIONS, STOP_SEQUENCES, ECHO_PROMPT, TOP_K_PER_TOKEN, MAX_TOKENS)

    print("Total prompts to query", total_prompts_to_query)

In [12]:
if __name__ == "__main__":
#     sys.argv[1:][0], sys.argv[1:][1]
    main('naturalqa', 'ada')

number of prefixes 51
number of questions read in 220
dataset naturalqa model name ada
output_file /u/scr/katezhou/Uncertainty/interpretability/naturalqa/results/10_1_openai_ada_gricean_experiment_qa.csv

template: 
len prompts to use 200
already queried 200
prompts to query 0
template:  It could be
len prompts to use 200
already queried 200
prompts to query 0
template:  Considering all the options it's
len prompts to use 200
already queried 200
prompts to query 0
template:  It probably is
len prompts to use 200
already queried 200
prompts to query 0
template:  Maybe it's
len prompts to use 200
already queried 200
prompts to query 0
template:  Perhaps it's
len prompts to use 200
already queried 200
prompts to query 0
template:  I don't know maybe it's
len prompts to use 200
already queried 200
prompts to query 0
template:  I suppose it's
len prompts to use 200
already queried 200
prompts to query 0
template:  I would need to double check but maybe it's
len prompts to use 200
already qu